<!-- <img src='./pic/SharifUniLogo.png' alt="SUT logo" width=300 height=300 align=left class="saturate" > <br> -->

<div dir=ltr align=center>
<font color=0F5298 size=7>
    Big Data - Massive Data Mining <br>
<font color=696080 size=5>
    Book Recommendation System <br>
<font color=696880 size=4>
    Mohammad Ali Etemadi Naeen
    
____

# **import Libraries**

In [3]:
!pip install pyspark
!pip install pyspark findspark
!pip install -U -q PyDrive2
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 124574 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Recommender_System") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

In [ ]:
import os
import re
import csv
import json
import string
import random
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler, VectorAssembler

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from mpl_toolkits.mplot3d import Axes3D

import hashlib
from hashlib import md5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.6 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


---
# **Q2**


### ```Pre-Processing```

In [ ]:
# Load the dataset into a DataFrame
file_path = "./data/BookRates_DS.csv"
data = spark.read.csv(file_path, header=False, inferSchema=True)

# Show the schema and a sample of the data to ensure it is loaded correctly
data.printSchema()
data.show(10)


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: integer (nullable = true)

+--------------+----------+---+----------+
|           _c0|       _c1|_c2|       _c3|
+--------------+----------+---+----------+
| AH2L9G3DQHHAJ|0000000116|4.0|1019865600|
|A2IIIDRK3PRRZY|0000000116|1.0|1395619200|
|A1TADCM7YWPQ8M|0000000868|4.0|1031702400|
| AWGH7V0BDOJKB|0000013714|4.0|1383177600|
|A3UTQPQPM4TQO0|0000013714|5.0|1374883200|
| A8ZS0I5L5V31B|0000013714|5.0|1393632000|
| ACNGUPJ3A3TM9|0000013714|4.0|1386028800|
|A3BED5QFJWK88M|0000013714|4.0|1350345600|
|A2SUAM1J3GNN3B|0000013714|5.0|1252800000|
| APOZ15IEYQRRR|0000013714|5.0|1362787200|
+--------------+----------+---+----------+
only showing top 10 rows



In [ ]:
data.count()

22502155

In [ ]:
# Step 1: Rename columns
data = data.withColumnRenamed("_c0", "user_id") \
           .withColumnRenamed("_c1", "book_id") \
           .withColumnRenamed("_c2", "rating") \
           .withColumnRenamed("_c3", "timestamp")

# Step 2: Check for null or duplicate values
data = data.dropDuplicates()
data = data.dropna()

# Show updated schema
data.printSchema()



root
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
data.show(10)

25/01/16 17:12:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:08 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------+----------+------+----------+
|       user_id|   book_id|rating| timestamp|
+--------------+----------+------+----------+
|A25DQKCOPNDCFC|000100039X|   5.0|1403049600|
| A2W6WXEUAVM3E|000100039X|   5.0|1046304000|
|A3GNW5MN7WN17J|000100039X|   5.0|1363392000|
| AXGNUEMHIDGNA|000100039X|   5.0|1377734400|
|A3AW2ZG0GP4SKN|000100039X|   5.0|1187827200|
|A3HQ42E4TQE2QY|0001473123|   1.0|1364774400|
|A1Z3P6UFJLTIF2|0001856871|   5.0|1336608000|
|A35VAQCQ4U28V7|0002007770|   2.0|1327363200|
|A2TXMYYGSZCSZ1|0002007770|   5.0|1235520000|
| ATOEBBOWB6ZQJ|0002007770|   2.0|1273708800|
+--------------+----------+------+----------+
only showing top 10 rows



In [ ]:
data

DataFrame[user_id: string, book_id: string, rating: double, timestamp: int]

In [ ]:
data.count()

25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:35 WARN RowBasedKeyValueBatch: Calling spill() on

22502155

In [ ]:
data = data.filter((data.rating >= 1) & (data.rating <= 5))
data.count()

25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:12:58 WARN RowBasedKeyValueBatch: Calling spill() on

22502155

In [ ]:
# data = data.withColumnRenamed("_c0", "user_id") \
#            .withColumnRenamed("_c1", "book_id") \
#            .withColumnRenamed("_c2", "rating") \
#            .withColumnRenamed("_c3", "timestamp")

In [ ]:
data = data.groupBy("user_id", "book_id").agg(
    F.mean("rating").alias("average_rating"),
    F.max("timestamp").alias("latest_timestamp")
)


In [ ]:
data.count()

25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:21 WARN RowBasedKeyValueBatch: Calling spill() on

[963.299s][warning][gc,alloc] Executor task launch worker for task 9.0 in stage 36.0 (TID 167): Retried waiting for GCLocker too often allocating 11597 words


25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/01/16 17:23:36 WARN RowBasedKeyValueBatch: Calling spill() on

22502155

---
### ```sampled dataset```

#### sampling

In [ ]:
file_path = "./data/BookRates_DS.csv"
data = pd.read_csv(file_path, header=None, names=['book_id', 'user_id', 'rating', 'timestamp'])

book_counts = data['book_id'].value_counts()
top_books = book_counts.head(1000).index

filtered_data = data[data['book_id'].isin(top_books)]

user_counts = filtered_data['user_id'].value_counts()
top_users = user_counts.head(1000).index

final_sample = filtered_data[filtered_data['user_id'].isin(top_users)]

final_sample.to_csv("./data/sampled_dataset2.csv", index=False)


In [ ]:
file_path = "./data/sampled_dataset.csv"
data = pd.read_csv(file_path, header=None, names=['book_id', 'user_id', 'rating', 'timestamp'])

unique_books = data['book_id'].nunique()
unique_users = data['user_id'].nunique()

print(f"Number of Distinct Books: {unique_books}")
print(f"Number of Distinct Users: {unique_users}")

Number of Distinct Books: 371
Number of Distinct Users: 401


In [ ]:
file_path = "./data/sampled_dataset2.csv"
data = pd.read_csv(file_path, header=None, names=['book_id', 'user_id', 'rating', 'timestamp'])

unique_books = data['book_id'].nunique()
unique_users = data['user_id'].nunique()

print(f"Number of Distinct Books: {unique_books}")
print(f"Number of Distinct Users: {unique_users}")

Number of Distinct Books: 936
Number of Distinct Users: 1001


In [ ]:
file_path = "./data/sampled_dataset3.csv"
data = pd.read_csv(file_path, header=None, names=['book_id', 'user_id', 'rating', 'timestamp'])

unique_books = data['book_id'].nunique()
unique_users = data['user_id'].nunique()

print(f"Number of Distinct Books: {unique_books}")
print(f"Number of Distinct Users: {unique_users}")

Number of Distinct Books: 9696
Number of Distinct Users: 10001


/tmp/ipykernel_1526/2992165164.py:2: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, header=None, names=['book_id', 'user_id', 'rating', 'timestamp'])


In [ ]:
# book_user_counts = data.groupby('book_id')['user_id'].nunique()
# book_user_counts.to_csv("book_user_counts.csv", header=True)

# user_book_counts = data.groupby('user_id')['book_id'].nunique()
# user_book_counts.to_csv("user_book_counts.csv", header=True)

#### starting the algorithm

In [ ]:
csv_path = './data/sampled_dataset.csv'
book_rates_rdd = sc.textFile(csv_path)

header = book_rates_rdd.first()
book_rates_rdd = book_rates_rdd.filter(lambda row: row != header)

book_rates_rdd = book_rates_rdd.map(lambda row: tuple(row.split(',')))
print(book_rates_rdd.take(5))

[('A30H2335OM7RD6', '0002007770', '5.0', '1230422400'), ('A21NVBFIEQWDSG', '0002007770', '5.0', '1187049600'), ('A2PWUIQG57O1AZ', '0002007770', '4.0', '1182124800'), ('A39650P2CZUUC9', '0002007770', '4.0', '1236124800'), ('A2F6N60Z96CAJI', '0002007770', '5.0', '1149033600')]


In [ ]:
book_rates_rdd.count()

9097

---

In [ ]:
book_rates_rdd = book_rates_rdd.map(lambda x: (x[0], x[1], float(x[2])))
print(book_rates_rdd.take(5))


[('A30H2335OM7RD6', '0002007770', 5.0), ('A21NVBFIEQWDSG', '0002007770', 5.0), ('A2PWUIQG57O1AZ', '0002007770', 4.0), ('A39650P2CZUUC9', '0002007770', 4.0), ('A2F6N60Z96CAJI', '0002007770', 5.0)]


In [ ]:
user_item_matrix = book_rates_rdd \
    .map(lambda x: (x[0], {x[1]: x[2]})) \
    .reduceByKey(lambda x, y: {**x, **y})

print(user_item_matrix.take(20))

[('A30H2335OM7RD6', {'0002007770': 5.0, '0060987103': 4.0, '0061009059': 5.0, '0099453924': 2.0, '0099464462': 5.0, '0140390464': 5.0, '0141039280': 5.0, '0261103288': 5.0, '0307245306': 5.0, '0312265840': 5.0, '0312265859': 5.0, '0312369727': 5.0, '0312675062': 5.0, '0385336667': 5.0, '0385340990': 5.0, '0399143793': 5.0, '0399147268': 5.0, '0399148612': 5.0}), ('A21NVBFIEQWDSG', {'0002007770': 5.0, '0007149824': 4.0, '0061147931': 5.0, '0099429799': 4.0, '0099513021': 4.0, '0141040343': 5.0, '0142004332': 5.0, '0143035304': 4.0, '0143037145': 5.0, '0143113496': 5.0, '015206396X': 5.0, '0307264556': 5.0, '0307275167': 4.0, '030727828X': 5.0, '0307279936': 4.0, '0307386171': 5.0, '0307387135': 4.0, '0312319169': 3.0, '0312347472': 4.0, '031242440X': 4.0, '0312426321': 5.0, '0316010669': 5.0, '0316011770': 4.0, '0316012823': 4.0, '0316069418': 5.0, '0316154741': 5.0, '0316346624': 4.0, '0340953950': 5.0, '0345476972': 4.0, '0345476999': 4.0, '0375411089': 4.0, '0385340990': 5.0, '038551

In [ ]:
book_features_rdd = book_rates_rdd \
    .map(lambda x: (x[1], (x[0], x[2]))) \
    .groupByKey() \
    .map(lambda x: (x[0], [rating for _, rating in x[1]]))



In [ ]:
# book_features_rdd = book_rates_rdd \
#     .map(lambda x: (x[0], (x[1], x[2]))) \
#     .groupByKey() \
#     .map(lambda x: (x[0], np.array([rating for _, rating in x[1]]).tolist()))


In [ ]:
book_features_rdd.take(2)

[('0002007770',
  [5.0,
   5.0,
   4.0,
   4.0,
   5.0,
   4.0,
   4.0,
   5.0,
   4.0,
   5.0,
   4.0,
   4.0,
   5.0,
   4.0,
   3.0,
   5.0,
   5.0,
   2.0,
   3.0,
   4.0,
   4.0,
   5.0,
   5.0,
   3.0,
   5.0,
   5.0,
   5.0,
   2.0,
   5.0,
   4.0,
   4.0,
   3.0,
   5.0,
   5.0,
   3.0,
   5.0,
   5.0]),
 ('0007124015',
  [3.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   4.0,
   2.0,
   5.0,
   5.0,
   5.0,
   2.0,
   5.0,
   5.0,
   5.0,
   5.0,
   4.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0,
   5.0])]

In [ ]:
# convert to DataFrame
book_features_df = book_features_rdd.toDF(["BookID", "Features"])
book_features_df.show(10)

+----------+--------------------+
|    BookID|            Features|
+----------+--------------------+
|0002007770|[5.0, 5.0, 4.0, 4...|
|0007124015|[3.0, 5.0, 5.0, 5...|
|0007136838|[4.0, 4.0, 5.0, 4...|
|0007167040|[5.0, 5.0, 5.0, 5...|
|0007172826|[5.0, 5.0, 4.0, 4...|
|0007386648|[5.0, 4.0, 5.0, 5...|
|0020519109|[5.0, 3.0, 5.0, 4...|
|0060098902|[5.0, 3.0, 5.0, 5...|
|0060175400|[5.0, 3.0, 5.0, 3...|
|0060175966|[3.0, 5.0, 4.0, 5...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
def generate_hash(features):
    features_str = ','.join(map(str, features))
    return hashlib.md5(features_str.encode()).hexdigest()

generate_hash_udf = udf(generate_hash, StringType())

book_features_df = book_features_df.withColumn("HashValues", generate_hash_udf(book_features_df["Features"]))

book_features_df.show()


+----------+--------------------+--------------------+
|    BookID|            Features|          HashValues|
+----------+--------------------+--------------------+
|0002007770|[5.0, 5.0, 4.0, 4...|594ea621dd3b3e1e2...|
|0007124015|[3.0, 5.0, 5.0, 5...|b691eb281616c24c6...|
|0007136838|[4.0, 4.0, 5.0, 4...|e60eb553d23fb8904...|
|0007167040|[5.0, 5.0, 5.0, 5...|ceba897e9dab7f03e...|
|0007172826|[5.0, 5.0, 4.0, 4...|ff4ddc68a09758962...|
|0007386648|[5.0, 4.0, 5.0, 5...|20eafd48b7b15e718...|
|0020519109|[5.0, 3.0, 5.0, 4...|ddffe0fef299932c8...|
|0060098902|[5.0, 3.0, 5.0, 5...|a0a23d229ad8dfbbd...|
|0060175400|[5.0, 3.0, 5.0, 3...|23484a21efeeb774e...|
|0060175966|[3.0, 5.0, 4.0, 5...|5d5c6a39417ec731b...|
|0060277270|[4.0, 5.0, 4.0, 5...|41ade9d435df61653...|
|0060530928|[5.0, 5.0, 5.0, 5...|6f12f4f1827dd1214...|
|0060558121|[2.0, 4.0, 3.0, 4...|b12168fe0fa210085...|
|0060572965|[4.0, 4.0, 5.0, 4...|11b5bc3af9d09e4aa...|
|0060572973|[5.0, 5.0, 3.0, 4...|27757bfe02881824f...|
|006058475

---
### ``` Pearson correlation coefficient```

In [ ]:
def pearson_correlation(features1, features2):
    n = len(features1)
    mean1 = sum(features1) / n
    mean2 = sum(features2) / n

    numerator = sum((x - mean1) * (y - mean2) for x, y in zip(features1, features2))
    denominator1 = sum((x - mean1) ** 2 for x in features1) ** 0.5
    denominator2 = sum((y - mean2) ** 2 for y in features2) ** 0.5
    denominator = denominator1 * denominator2

    return numerator / denominator if denominator != 0 else 0.0

pearson_correlation_udf = udf(pearson_correlation, FloatType())
cartesian_df = book_features_df.alias("A").crossJoin(book_features_df.alias("B"))

pearson_similar_items = cartesian_df.withColumn(
    "Similarity",
    pearson_correlation_udf(col("A.Features"), col("B.Features"))
).filter(col("A.BookID") != col("B.BookID"))


In [ ]:
pearson_similar_items.show()

+----------+--------------------+--------------------+----------+--------------------+--------------------+-------------+
|    BookID|            Features|          HashValues|    BookID|            Features|          HashValues|   Similarity|
+----------+--------------------+--------------------+----------+--------------------+--------------------+-------------+
|0002007770|[5.0, 5.0, 4.0, 4...|594ea621dd3b3e1e2...|0007124015|[3.0, 5.0, 5.0, 5...|b691eb281616c24c6...|  -0.26486424|
|0002007770|[5.0, 5.0, 4.0, 4...|594ea621dd3b3e1e2...|0007136838|[4.0, 4.0, 5.0, 4...|e60eb553d23fb8904...|   0.11497497|
|0002007770|[5.0, 5.0, 4.0, 4...|594ea621dd3b3e1e2...|0007167040|[5.0, 5.0, 5.0, 5...|ceba897e9dab7f03e...|  -0.02273257|
|0002007770|[5.0, 5.0, 4.0, 4...|594ea621dd3b3e1e2...|0007172826|[5.0, 5.0, 4.0, 4...|ff4ddc68a09758962...|   0.17255066|
|0002007770|[5.0, 5.0, 4.0, 4...|594ea621dd3b3e1e2...|0007386648|[5.0, 4.0, 5.0, 5...|20eafd48b7b15e718...|-0.0045036594|
|0002007770|[5.0, 5.0, 4

In [ ]:
pearson_similar_items = pearson_similar_items.select(
    col("A.BookID").alias("BookID1"),
    col("B.BookID").alias("BookID2"),
    "Similarity"
)

pearson_similar_items.show()

+----------+----------+-------------+
|   BookID1|   BookID2|   Similarity|
+----------+----------+-------------+
|0002007770|0007124015|  -0.26486424|
|0002007770|0007136838|   0.11497497|
|0002007770|0007167040|  -0.02273257|
|0002007770|0007172826|   0.17255066|
|0002007770|0007386648|-0.0045036594|
|0002007770|0020519109| -0.050422482|
|0002007770|0060098902| -0.040890608|
|0002007770|0060175400|  -0.11195717|
|0002007770|0060175966|  -0.06600658|
|0002007770|0060277270|   0.05727312|
|0002007770|0060530928| -0.020993248|
|0002007770|0060558121|  0.032906815|
|0002007770|0060572965|  -0.04903344|
|0002007770|0060572973|   0.07296951|
|0002007770|0060584750|  -0.06422061|
|0002007770|006073132X| -0.029173234|
|0002007770|0060838582|   -0.0982114|
|0002007770|0060850523| -0.060891442|
|0002007770|0060872985| -0.017387502|
|0002007770|0060987103|   0.15237081|
+----------+----------+-------------+
only showing top 20 rows



In [ ]:
user_item_matrix.take(10)


[('A30H2335OM7RD6',
  {'0002007770': 5.0,
   '0060987103': 4.0,
   '0061009059': 5.0,
   '0099453924': 2.0,
   '0099464462': 5.0,
   '0140390464': 5.0,
   '0141039280': 5.0,
   '0261103288': 5.0,
   '0307245306': 5.0,
   '0312265840': 5.0,
   '0312265859': 5.0,
   '0312369727': 5.0,
   '0312675062': 5.0,
   '0385336667': 5.0,
   '0385340990': 5.0,
   '0399143793': 5.0,
   '0399147268': 5.0,
   '0399148612': 5.0}),
 ('A21NVBFIEQWDSG',
  {'0002007770': 5.0,
   '0007149824': 4.0,
   '0061147931': 5.0,
   '0099429799': 4.0,
   '0099513021': 4.0,
   '0141040343': 5.0,
   '0142004332': 5.0,
   '0143035304': 4.0,
   '0143037145': 5.0,
   '0143113496': 5.0,
   '015206396X': 5.0,
   '0307264556': 5.0,
   '0307275167': 4.0,
   '030727828X': 5.0,
   '0307279936': 4.0,
   '0307386171': 5.0,
   '0307387135': 4.0,
   '0312319169': 3.0,
   '0312347472': 4.0,
   '031242440X': 4.0,
   '0312426321': 5.0,
   '0316010669': 5.0,
   '0316011770': 4.0,
   '0316012823': 4.0,
   '0316069418': 5.0,
   '03161547

---

### ``` Predict Score```

In [ ]:
# Mean on Train data
mean_rating = book_rates_rdd.map(lambda x: float(x[2])).mean()

# user-bias
user_bias = book_rates_rdd \
    .map(lambda x: (x[0], (float(x[2]) - mean_rating))) \
    .groupByKey() \
    .mapValues(lambda values: sum(values) / len(values)) \
    .collectAsMap()

# item-bias
item_bias = book_rates_rdd \
    .map(lambda x: (x[1], (float(x[2]) - mean_rating - user_bias.get(x[0], 0)))) \
    .groupByKey() \
    .mapValues(lambda values: sum(values) / len(values)) \
    .collectAsMap()

In [ ]:
def predict_rating(user, book, user_item_matrix, similar_items, mean_rating, user_bias, item_bias):

    baseline = mean_rating + user_bias.get(user, 0) + item_bias.get(book, 0)

    similar_books = similar_items.filter(
        (col("BookID1") == book) | (col("BookID2") == book)
    ).rdd.map(
        lambda x: (x["BookID2"] if x["BookID1"] == book else x["BookID1"], x["Similarity"])
    )

    numerator = 0
    denominator = 0
    for similar_book, similarity in similar_books.collect():
        if user in user_item_matrix and similar_book in user_item_matrix[user]:
            if similarity > 0:
                numerator += similarity * (user_item_matrix[user][similar_book] - baseline)
                denominator += abs(similarity)

    predicted_rating = baseline + (numerator / denominator if denominator != 0 else 0)

    # predicted_rating = max(1, min(5, predicted_rating))

    return predicted_rating


example_user_1 = "A30H2335OM7RD6"
example_book_1 = "0007136838"
predicted_rating_1 = predict_rating(example_user_1, example_book_1, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_1} on Book {example_book_1}: {predicted_rating_1}")

example_user_2 = "A20EEWWSFMZ1PN"
example_book_2 = "0002007770"
predicted_rating_2 = predict_rating(example_user_2, example_book_2, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_2} on Book {example_book_2}: {predicted_rating_2}")

example_user_3 = "243252fw"
example_book_3 = "w2"
predicted_rating_3 = predict_rating(example_user_3, example_book_3, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_3} on Book {example_book_3}: {predicted_rating_3}")


Predicted rating for User A30H2335OM7RD6 on Book 0007136838: 4.381743995015661
Predicted rating for User A20EEWWSFMZ1PN on Book 0002007770: 4.804942576281027
Predicted rating for User 243252fw on Book w2: 4.268440145102783


In [ ]:
example_user_3 = "AWFMGDUUGNGYW"
example_book_3 = "1594744769"
predicted_rating_3 = predict_rating(example_user_3, example_book_3, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_3} on Book {example_book_3}: {predicted_rating_3}")

Predicted rating for User AWFMGDUUGNGYW on Book 1594744769: 4.0364967281058135


In [ ]:
example_user_3 = "A2VMT89TCSF105"
example_book_3 = "1594744769"
predicted_rating_3 = predict_rating(example_user_3, example_book_3, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_3} on Book {example_book_3}: {predicted_rating_3}")

Predicted rating for User A2VMT89TCSF105 on Book 1594744769: 3.263675662854183


In [ ]:
users = book_rates_rdd.map(lambda x: x[0]).distinct().collect()
books = book_rates_rdd.map(lambda x: x[1]).distinct().collect()

In [ ]:
low_rating_combinations = []

for user in users:
    for book in books:
        predicted_rating = predict_rating(user, book, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
        if predicted_rating < 4:
            low_rating_combinations.append((user, book, predicted_rating))
            if len(low_rating_combinations) == 30:
                break
    if low_rating_combinations:
        break

In [ ]:
for user, book, rating in low_rating_combinations:
    print(f"User: {user}, Book: {book}, Predicted Rating: {rating}")

User: A30H2335OM7RD6, Book: 0060558121, Predicted Rating: 3.7334897413755757
User: A30H2335OM7RD6, Book: 0062015664, Predicted Rating: 3.5418788778777577
User: A30H2335OM7RD6, Book: 0062049801, Predicted Rating: 3.506371265036087
User: A30H2335OM7RD6, Book: 0062071246, Predicted Rating: 3.9417171985581354
User: A30H2335OM7RD6, Book: 0141034262, Predicted Rating: 3.962309987627539
User: A30H2335OM7RD6, Book: 0142004030, Predicted Rating: 3.8953876805614556
User: A30H2335OM7RD6, Book: 0143035304, Predicted Rating: 3.9975180843445224
User: A30H2335OM7RD6, Book: 0143113496, Predicted Rating: 3.6594994945628927
User: A30H2335OM7RD6, Book: 0307885151, Predicted Rating: 3.952518923412303
User: A30H2335OM7RD6, Book: 0385340990, Predicted Rating: 3.9303305082024487
User: A30H2335OM7RD6, Book: 0743496728, Predicted Rating: 3.210288994720364
User: A30H2335OM7RD6, Book: 147780756X, Predicted Rating: 3.035498712282858
User: A30H2335OM7RD6, Book: 0060530677, Predicted Rating: 3.8686216535129763
User

In [ ]:
lower_rating_combinations = []

for user in users:
    for book in books:
        predicted_rating = predict_rating(user, book, user_item_matrix.collectAsMap(), pearson_similar_items, mean_rating, user_bias, item_bias)
        if predicted_rating < 3:
            lower_rating_combinations.append((user, book, predicted_rating))
            if len(lower_rating_combinations) == 1:
                break
    if lower_rating_combinations:
        break


In [ ]:
for user, book, rating in lower_rating_combinations:
    print(f"User: {user}, Book: {book}, Predicted Rating: {rating}")

User: A2ZHH7AK83JB5G, Book: 0441015891, Predicted Rating: 2.663751061003054


In [ ]:
users

['A30H2335OM7RD6',
 'A21NVBFIEQWDSG',
 'A2PWUIQG57O1AZ',
 'A1T17LMQABMBN5',
 'A1K1JW1C5CUSUZ',
 'AWFMGDUUGNGYW',
 'A3PGGI7A6XCNF1',
 'A2E3GFHUDNPYDH',
 'A22B13EJTZKCTM',
 'A2MF2QVSCUI27G',
 'A1RMHZSWZ7ZEQO',
 'AS6SH2JM2YY3H',
 'A2ZHH7AK83JB5G',
 'A1S3IN6CSTBPY6',
 'A2VMT89TCSF105',
 'A20IIR0422G3A5',
 'A11B61QBGHLQDN',
 'A3GQKB1KF0CRPE',
 'A1D2C0WDCSHUWZ',
 'AQZH7YTWQPOBE',
 'A3FHAMLNGHCFU7',
 'A1E1LEVQ9VQNK',
 'AKT8TGIT6VVZ5',
 'AOKGRSAS5VNS3',
 'A14OJS0VWMOSWO',
 'A2F3M93RRLFQNJ',
 'A3QVAKVRAH657N',
 'A2B8GXSCB1R05T',
 'A1RAUVCWYHTQI4',
 'AISUSPFYY68RO',
 'A1MC6BFHWY6WC3',
 'A3A6GXILVT9JNW',
 'A2O9HU1K7TCF7B',
 'A2AYSFGUP5VTY3',
 'AN8C7BGR98HLN',
 'AHUT55E980RDR',
 'A3I8FT707XKQAJ',
 'A2B1GZIRD6W646',
 'A1RY8R18NEO9AA',
 'A2UIXU97JYCPZG',
 'A8IPQ1Q1O7YX5',
 'AHD101501WCN1',
 'A2HM0BZWQRV1EF',
 'A13OFOB1394G31',
 'A14BTJRH9VNLJJ',
 'A29NUB3P6YIWZG',
 'A1MJMYLRTZ76ZX',
 'A34UTL4AVX80MK',
 'A2YXRT2XIJIO57',
 'AN0XWUHSHRUG6',
 'A2NJO6YE954DBH',
 'A1L43KWWR05PCS',
 'A319KYEIAZ3SON',
 'AUM

---

### ``` RMSE Calculation```


In [ ]:
train_rdd, test_rdd = book_rates_rdd.randomSplit([0.8, 0.2], seed=42)

test_data = test_rdd.map(lambda x: (x[0], x[1], x[2]))


In [ ]:
user_item_matrix_dict = user_item_matrix.collectAsMap()
user_item_matrix_broadcast = sc.broadcast(user_item_matrix_dict)

train_book_features_rdd = train_rdd \
    .map(lambda x: (x[1], (x[0], x[2]))) \
    .groupByKey() \
    .map(lambda x: (x[0], [rating for _, rating in x[1]]))


train_book_features_df = train_book_features_rdd.toDF(["BookID", "Features"])

train_cartesian_df = train_book_features_df.alias("A").crossJoin(train_book_features_df.alias("B"))

similar_items_train = train_cartesian_df.withColumn(
    "Similarity",
    pearson_correlation_udf(col("A.Features"), col("B.Features"))
).filter(col("A.BookID") != col("B.BookID")) \
 .select(
    col("A.BookID").alias("BookID1"),
    col("B.BookID").alias("BookID2"),
    "Similarity"
)


similar_items_dict = similar_items_train.rdd.map(
    lambda row: (row["BookID1"], (row["BookID2"], row["Similarity"]))
).groupByKey().mapValues(list).collectAsMap()

similar_items_broadcast = sc.broadcast(similar_items_dict)


def predict_rating(user, book, user_item_matrix_broadcast, similar_items_broadcast):

    user_item_matrix = user_item_matrix_broadcast.value
    similar_items = similar_items_broadcast.value

    similar_books = similar_items.get(book, [])

    numerator = 0
    denominator = 0

    for similar_book, similarity in similar_books:
        if user in user_item_matrix and similar_book in user_item_matrix[user]:
            numerator += similarity * user_item_matrix[user][similar_book]
            denominator += abs(similarity)

    return numerator / denominator if denominator != 0 else 0


def compute_rmse(test_data, user_item_matrix_broadcast, similar_items_broadcast):
    def calculate_error(row):
        user, book, actual_rating = row
        predicted_rating = predict_rating(user, book, user_item_matrix_broadcast, similar_items_broadcast)
        return (predicted_rating - actual_rating) ** 2

    errors = test_data.map(calculate_error)
    mean_squared_error = errors.mean()
    rmse = mean_squared_error ** 0.5
    return rmse

rmse = compute_rmse(test_data, user_item_matrix_broadcast, similar_items_broadcast)
print(f"Root Mean Squared Error (RMSE): {rmse}")


Root Mean Squared Error (RMSE): 4.399588671397565


#### Adding Bias to Reduce the RMSE

In [ ]:
user_item_matrix_dict = user_item_matrix.collectAsMap()
user_item_matrix_broadcast = sc.broadcast(user_item_matrix_dict)

train_book_features_rdd = train_rdd \
    .map(lambda x: (x[1], (x[0], x[2]))) \
    .groupByKey() \
    .map(lambda x: (x[0], [rating for _, rating in x[1]]))


train_book_features_df = train_book_features_rdd.toDF(["BookID", "Features"])

train_cartesian_df = train_book_features_df.alias("A").crossJoin(train_book_features_df.alias("B"))

similar_items_train = train_cartesian_df.withColumn(
    "Similarity",
    pearson_correlation_udf(col("A.Features"), col("B.Features"))
).filter(col("A.BookID") != col("B.BookID")) \
 .select(
    col("A.BookID").alias("BookID1"),
    col("B.BookID").alias("BookID2"),
    "Similarity"
)


similar_items_dict = similar_items_train.rdd.map(
    lambda row: (row["BookID1"], (row["BookID2"], row["Similarity"]))
).groupByKey().mapValues(list).collectAsMap()

similar_items_broadcast = sc.broadcast(similar_items_dict)


global_mean = sum([sum(ratings.values()) for ratings in user_item_matrix_dict.values()]) / \
              sum([len(ratings) for ratings in user_item_matrix_dict.values()])


user_bias = {user: (sum(ratings.values()) / len(ratings) - global_mean)
             for user, ratings in user_item_matrix_dict.items()}

book_bias = {}
for user, ratings in user_item_matrix_dict.items():
    for book, rating in ratings.items():
        book_bias[book] = book_bias.get(book, []) + [rating]

book_bias = {book: (sum(ratings) / len(ratings) - global_mean) for book, ratings in book_bias.items()}

def predict_rating(user, book, user_item_matrix_broadcast, similar_items_broadcast):
    user_item_matrix = user_item_matrix_broadcast.value
    similar_items = similar_items_broadcast.value

    similar_books = similar_items.get(book, [])

    numerator = 0
    denominator = 0

    for similar_book, similarity in similar_books:
        if user in user_item_matrix and similar_book in user_item_matrix[user]:
            numerator += similarity * user_item_matrix[user][similar_book]
            denominator += abs(similarity)

    bias = global_mean + user_bias.get(user, 0) + book_bias.get(book, 0)
    prediction = bias + (numerator / denominator if denominator != 0 else 0)

    return prediction

def compute_rmse(test_data, user_item_matrix_broadcast, similar_items_broadcast):
    def calculate_error(row):
        user, book, actual_rating = row
        predicted_rating = predict_rating(user, book, user_item_matrix_broadcast, similar_items_broadcast)
        return (predicted_rating - actual_rating) ** 2

    errors = test_data.map(calculate_error)
    mean_squared_error = errors.mean()
    rmse = mean_squared_error ** 0.5
    return rmse

rmse = compute_rmse(test_data, user_item_matrix_broadcast, similar_items_broadcast)
print(f"Root Mean Squared Error (RMSE): {rmse}")


Root Mean Squared Error (RMSE): 1.8955287916243304


The RMSE value of **1.895** is relatively acceptable compared to the rating scale (**1 to 5**). However, there is still room for further improvement.

Strengths of the Current Model are:
- **Incorporating Biases**: Considering biases has resulted in better modeling of baseline ratings.
- **Feature-Based Similarity**: Using feature-based similarity for books has allowed for relatively more accurate computation of item relationships.


#### ```improve RMSE```
To improve the results, both **weight normalization** and **adding regularization** can be applied.

 1. **Weight Normalization**
- **Objective**: Enhance the impact of stronger similarities on predictions while reducing the influence of weaker similarities.
- **Implementation**:
  - Assign a minimum threshold to similarities and ignore those below it.
  - Normalize similarities so their sum equals \(1\).


 2. **Adding Regularization**
- **Objective**: Reduce the impact of overfitting when calculating biases and similarities.
- **Implementation**:
  - Compute user and book biases by adding a regularization term λ:
    $$
    b_u = \frac{\sum_{i \in I_u} (r_{ui} - \mu)}{\lambda + |I_u|}
    $$
  - λ is the regularization term, which balances the model.



In [ ]:
similar_items_dict = similar_items_train.rdd.map(
    lambda row: (row["BookID1"], (row["BookID2"], row["Similarity"]))
).groupByKey().mapValues(list).collectAsMap()

similar_items_broadcast = sc.broadcast(similar_items_dict)

In [ ]:
global_mean = train_rdd.map(lambda x: x[2]).mean()


user_bias = train_rdd.map(lambda x: (x[0], x[2] - global_mean)) \
    .groupByKey() \
    .mapValues(lambda ratings: sum(ratings) / len(ratings)) \
    .collectAsMap()

item_bias = train_rdd.map(lambda x: (x[1], x[2] - global_mean)) \
    .groupByKey() \
    .mapValues(lambda ratings: sum(ratings) / len(ratings)) \
    .collectAsMap()

user_bias_broadcast = sc.broadcast(user_bias)
item_bias_broadcast = sc.broadcast(item_bias)

def predict_rating_with_bias(user, book, similar_items_broadcast, user_item_matrix_broadcast, user_bias_broadcast, item_bias_broadcast, lambda_reg=0.05):
    user_item_matrix = user_item_matrix_broadcast.value
    similar_items = similar_items_broadcast.value.get(book, [])
    user_bias = user_bias_broadcast.value
    item_bias = item_bias_broadcast.value

    user_b = user_bias.get(user, 0)
    item_b = item_bias.get(book, 0)

    numerator = 0
    denominator = 0

    total_similarity = sum(similarity for _, similarity in similar_items)
    normalized_similar_books = [(similar_book, similarity / total_similarity) for similar_book, similarity in similar_items if total_similarity > 0]

    for similar_book, similarity in normalized_similar_books:
        if user in user_item_matrix and similar_book in user_item_matrix[user]:
            rating = user_item_matrix[user][similar_book]
            numerator += similarity * rating
            denominator += abs(similarity)

    predicted_interaction = numerator / denominator if denominator != 0 else 0
    regularized_bias = lambda_reg / (lambda_reg + len(user_item_matrix.get(user, {})))

    return global_mean + user_b + item_b + predicted_interaction + regularized_bias

def compute_rmse_with_bias(test_data, user_item_matrix_broadcast, similar_items_broadcast, user_bias_broadcast, item_bias_broadcast):
    def calculate_error(row):
        user, book, actual_rating = row
        predicted_rating = predict_rating_with_bias(
            user, book, similar_items_broadcast, user_item_matrix_broadcast, user_bias_broadcast, item_bias_broadcast
        )
        return (predicted_rating - actual_rating) ** 2

    errors = test_data.map(calculate_error)
    mean_squared_error = errors.mean()
    rmse = mean_squared_error ** 0.5
    return rmse

rmse_with_bias = compute_rmse_with_bias(test_data, user_item_matrix_broadcast, similar_items_broadcast, user_bias_broadcast, item_bias_broadcast)
print(f"Root Mean Squared Error (RMSE) with Bias: {rmse_with_bias}")


Root Mean Squared Error (RMSE) with Bias: 1.6184307157320332


---

### ```Best RMSE```

$$
\text{RMSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2}
$$

- yi: Real Score
- ŷi: Predicted Score
- N: Samples in Test Data


In [ ]:
train_data, test_data = book_rates_rdd.randomSplit([0.8, 0.2], seed=42)

print(f"Train set size: {train_data.count()}")
print(f"Test set size: {test_data.count()}")

Train set size: 7271
Test set size: 1826


In [ ]:
# Mean on Train data
mean_rating = train_data.map(lambda x: float(x[2])).mean()

# user-bias on Train data
user_bias = train_data \
    .map(lambda x: (x[0], (float(x[2]) - mean_rating))) \
    .groupByKey() \
    .mapValues(lambda values: sum(values) / len(values)) \
    .collectAsMap()

# item-bias on Train data
item_bias = train_data \
    .map(lambda x: (x[1], (float(x[2]) - mean_rating - user_bias.get(x[0], 0)))) \
    .groupByKey() \
    .mapValues(lambda values: sum(values) / len(values)) \
    .collectAsMap()

In [ ]:
user_item_matrix_dict = user_item_matrix.collectAsMap()

pearson_similar_items_dict = pearson_similar_items.rdd.map(lambda x: (
    (x["BookID1"], x["BookID2"]), x["Similarity"]
)).collectAsMap()

broadcast_user_item_matrix = sc.broadcast(user_item_matrix_dict)
broadcast_pearson_similar_items = sc.broadcast(pearson_similar_items_dict)

In [ ]:
def predict_rating(user, book, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias):

    baseline = mean_rating + user_bias.get(user, 0) + item_bias.get(book, 0)

    similar_books = []
    for (book1, book2), similarity in pearson_similar_items_dict.items():
        if book1 == book:
            similar_books.append((book2, similarity))
        elif book2 == book:
            similar_books.append((book1, similarity))

    numerator = 0
    denominator = 0
    for similar_book, similarity in similar_books:
        if user in user_item_matrix_dict and similar_book in user_item_matrix_dict[user]:
            if similarity > 0:
                numerator += similarity * (user_item_matrix_dict[user][similar_book] - baseline)
                denominator += abs(similarity)

    predicted_rating = baseline + (numerator / denominator if denominator != 0 else 0)

    # predicted_rating = max(1, min(5, predicted_rating))

    return predicted_rating

In [ ]:
example_user_1 = "A30H2335OM7RD6"
example_book_1 = "0007136838"
predicted_rating_1 = predict_rating(example_user_1, example_book_1, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_1} on Book {example_book_1}: {predicted_rating_1}")

example_user_2 = "A20EEWWSFMZ1PN"
example_book_2 = "0002007770"
predicted_rating_2 = predict_rating(example_user_2, example_book_2, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_2} on Book {example_book_2}: {predicted_rating_2}")

example_user_3 = "243252fw"
example_book_3 = "w2"
predicted_rating_3 = predict_rating(example_user_3, example_book_3, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)
print(f"Predicted rating for User {example_user_3} on Book {example_book_3}: {predicted_rating_3}")


Predicted rating for User A30H2335OM7RD6 on Book 0007136838: 4.381743995015661
Predicted rating for User A20EEWWSFMZ1PN on Book 0002007770: 4.804942576281027
Predicted rating for User 243252fw on Book w2: 4.266400770182919


In [ ]:
from math import sqrt

test_predictions = test_data.map(lambda x: (
    x[0],  # User ID
    x[1],  # Book ID
    float(x[2]),  # Actual Rating
    predict_rating(x[0], x[1], user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)  # Predicted Rating
))

squared_errors = test_predictions.map(lambda x: (x[2] - x[3]) ** 2)

mse = squared_errors.mean()
rmse = sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse}")

RMSE: 0.8630944237430908


In [ ]:
print(f"Root Mean Squared Error (RMSE): 0.8630944237430908")

**Interpreting the RMSE Value**

1. **RMSE Value**:
   - The lower the RMSE value, the more accurate model.
   - In recommender systems, RMSE values below 1 typically indicate good performance.

2. **Comparison with the Rating Scale**:
   - Since the rating scale is between 1 and 5, an error of **0.86** means that predictions are, on average, less than 1 unit away from the actual ratings. So This is acceptable for a recommender system.


---

### ```Fill the Evaluation File with Rating```

In [ ]:
train_data, test_data = book_rates_rdd.randomSplit([0.8, 0.2], seed=42)

print(f"Train set size: {train_data.count()}")
print(f"Test set size: {test_data.count()}")

Train set size: 7271
Test set size: 1826


In [ ]:
# Mean on Train data
mean_rating = train_data.map(lambda x: float(x[2])).mean()

# user-bias on Train data
user_bias = train_data \
    .map(lambda x: (x[0], (float(x[2]) - mean_rating))) \
    .groupByKey() \
    .mapValues(lambda values: sum(values) / len(values)) \
    .collectAsMap()

# item-bias on Train data
item_bias = train_data \
    .map(lambda x: (x[1], (float(x[2]) - mean_rating - user_bias.get(x[0], 0)))) \
    .groupByKey() \
    .mapValues(lambda values: sum(values) / len(values)) \
    .collectAsMap()

In [ ]:
user_item_matrix_dict = user_item_matrix.collectAsMap()

pearson_similar_items_dict = pearson_similar_items.rdd.map(lambda x: (
    (x["BookID1"], x["BookID2"]), x["Similarity"]
)).collectAsMap()

broadcast_user_item_matrix = sc.broadcast(user_item_matrix_dict)
broadcast_pearson_similar_items = sc.broadcast(pearson_similar_items_dict)

In [ ]:
def predict_rating(user, book, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias):

    baseline = mean_rating + user_bias.get(user, 0) + item_bias.get(book, 0)

    similar_books = []
    for (book1, book2), similarity in pearson_similar_items_dict.items():
        if book1 == book:
            similar_books.append((book2, similarity))
        elif book2 == book:
            similar_books.append((book1, similarity))

    numerator = 0
    denominator = 0
    for similar_book, similarity in similar_books:
        if user in user_item_matrix_dict and similar_book in user_item_matrix_dict[user]:
            if similarity > 0:
                numerator += similarity * (user_item_matrix_dict[user][similar_book] - baseline)
                denominator += abs(similarity)

    predicted_rating = baseline + (numerator / denominator if denominator != 0 else 0)

    # predicted_rating = max(1, min(5, predicted_rating))

    return predicted_rating


input_file = './data/Evaluate_P2.csv'
output_file = './output/Evaluation_P2_Completed.csv'

user_item_matrix_dict = user_item_matrix.collectAsMap()
pearson_similar_items_dict = pearson_similar_items.rdd.map(lambda x: (
    (x["BookID1"], x["BookID2"]), x["Similarity"]
)).collectAsMap()

with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = next(reader)
    writer.writerow(header)

    for row in reader:
        user_id = row[0]
        book_id = row[1]

        # Calculate Predict Score
        predicted_rating = predict_rating(user_id, book_id, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)

        # Replace the Predicted Score with ?
        row[2] = str(predicted_rating)

        writer.writerow(row)


---

### ```Recommend Books```

In [ ]:
def recommend_books_based_on_predictions(user_id, book_rates_rdd, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias):

    all_books = book_rates_rdd.map(lambda x: x[1]).distinct().collect()

    user_rated_books = book_rates_rdd.filter(lambda x: x[0] == user_id).map(lambda x: x[1]).collect()

    unrated_books = [book for book in all_books if book not in user_rated_books]

    if not unrated_books:
        print(f"User {user_id} rated all books")
        return

    predictions = []
    for book in unrated_books:
        predicted_rating = predict_rating(user_id, book, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)
        predictions.append((book, predicted_rating))

    sorted_by_lowest = sorted(predictions, key=lambda x: x[1])
    sorted_by_highest = sorted(predictions, key=lambda x: x[1], reverse=True)

    lowest_rated_books = sorted_by_lowest[:3]
    highest_rated_books = sorted_by_highest[:3]

    print(f"User {user_id}:")
    print("\n 3 Lowest Predicted Scores:")
    for book, rating in lowest_rated_books:
        print(f"Book ID: {book}, Predicted Rating: {rating}")

    print("\n 3 Highest Predicted Scores:")
    for book, rating in highest_rated_books:
        print(f"Book ID: {book}, Predicted Rating: {rating}")

# User ID
example_user = "A30H2335OM7RD6"
recommend_books_based_on_predictions(example_user, book_rates_rdd, user_item_matrix_dict, pearson_similar_items_dict, mean_rating, user_bias, item_bias)

User A30H2335OM7RD6:

 3 Lowest Predicted Scores:
Book ID: 147780756X, Predicted Rating: 3.035498712282858
Book ID: 0743566572, Predicted Rating: 3.189187019563758
Book ID: 0743496728, Predicted Rating: 3.210288994720364

 3 Highest Predicted Scores:
Book ID: 0020519109, Predicted Rating: 5.0
Book ID: 0060098902, Predicted Rating: 5.0
Book ID: 0060175400, Predicted Rating: 5.0
